In [2]:
#Do this before starting the jupyter server
!export PYTHONUSERBASE=$HOME/.usr/local/python/2.7.13

In [3]:
#TODO: When movin to singularity, try to get better version of dlib/ face_recognition installed
import os
import face_recognition
import numpy as np
import pandas as pd 
import pickle
from os.path import join
from tqdm import tqdm

def load_pickle(filename):
    infile = open(filename,'rb')
    x = pickle.load(infile)
    infile.close()
    return x

def store_pickle(filename, data):
    file = open(filename,'wb')
    pickle.dump(data, file)
    file.close()

#TODO: read the csv and rename all directories with their celeb name   

In [3]:
#TODO:    1. Rename directories
#         2. get list of people who are news reporters/anchors + maybe failed?
#         3. Image extraction - get more than 10 images per person in the list
#         4. Extract encodings of only these people
#         5. Then fit into the KNN function

In [4]:
#Screen instructions:
#start the VPN
#login to rider.case.edu; on login, use $screen
#load the modules and start jupyter server
#get a local GUI using ssh -N -L, put the code on execution
#quit the ssh -N -L terminal
#ctrl+A, ctrl+D on the screen'd terminal, to exit the screen
#Logout from the cluster
#turn off VPN

In [3]:
!ls

anchors-with-TV-encodings.pickle  insightface		  pickles
deepInsight.ipynb		  MS_Celeb_cropped	  pyth_modules_dir
encodings_test.ipynb		  MS_Celeb_uncropped	  tsv_extractor.py
failed.pickle			  one_time_changes.ipynb


In [4]:
root = '/home/skb78/gallinahome/MS_Celeb_uncropped/MS-Celeb-1M/data/croped_face_images/extracted_images/'
output_path = 'anchors-with-TV-encodings.pickle'
# celebs = sorted(os.listdir(root))[1:]
_, celebs = load_pickle('MS_Celeb_uncropped/anchors-anchors_with_TV-tuple.pickle')
celebs = list(celebs['name'])
encodings = []
face_not_found = []
failed_to_load = []
for i,celeb in enumerate(tqdm(celebs)):
        
    image_paths = os.listdir(join(root,celeb))[:10] #take encodings only for 10 images per celeb
    encodings.append([])
    for path in image_paths:
        img_location = join(root, celeb, path)
        try:
            img = face_recognition.load_image_file(img_location)
        except:
            failed_to_load.append(img_location)
#             print('failed to load for {}/{}'.format(celeb,path))
            continue
        
        #TEST: if bboxes too close to boundaries, use bboxes.txt
        bboxes = face_recognition.face_locations(img,model='hog') #TODO: Test if works well else take cnn
        if len(bboxes)>1:
            bboxes = [max(bboxes, key=lambda x: ((x[2]-x[0])*(x[1]-x[3])))] #If multiple faces are found, we take the largest face
#             print(bboxes)
        elif len(bboxes)==0:
            face_not_found.append(img_location)
#             print('face not found for {}/{}'.format(celeb,path))
            continue
        
        enc = face_recognition.face_encodings(img, bboxes)
        encodings[-1].append(enc[0])

encodings = np.array(encodings)
store_pickle(output_path, (celebs, encodings))
store_pickle('failed.pickle', (face_not_found, failed_to_load))
#TODO: check not_found after done
#TODO if method isn't satisfactory
#     1. More face images per celeb
#     2. Play around with K in KNN
#     3. CNN instead of hog

 19%|█▉        | 3024/15830 [2:39:22<6:03:04,  1.70s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 69%|██████▉   | 10996/15830 [6:22:48<2:14:04,  1.66s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 15830/15830 [8:37:47<00:00,  1.68s/it]


In [6]:
# encodings = np.array(encodings)
# store_pickle('9873outOf99891-encodings.pickle', (celebs, encodings))
# store_pickle('9873outOf99891-failed.pickle', (face_not_found, failed_to_load))

In [18]:
celebs, encodings = load_pickle('anchors-with-TV-encodings.pickle')

In [39]:
encodings = np.array([np.array(x) for x in encodings])

In [46]:
#TODO: KNN (X)
# convert (celebs, encodings) data into the format required by KNN
# Fill KNN space with encodings of all celebs before looping over all faces
# def face2name(face_img):
    # get encodings of face_img
    # pass it through KNN and infer
    # return the found name

# Populating KNN space with labelled encodings
from sklearn.neighbors import KNeighborsClassifier
X = []
Y = []
for i in range(len(encodings)): #prepare dataset
    for encoding in encodings[i]:
        X.append(encoding)
        Y.append(celebs[i])
        
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [48]:
Y

[u'Amy Grant',
 u'Amy Grant',
 u'Amy Grant',
 u'Amy Grant',
 u'Amy Grant',
 u'Amy Grant',
 u'Amy Grant',
 u'Amy Grant',
 u'Amy Grant',
 u'Adam Carolla',
 u'Adam Carolla',
 u'Adam Carolla',
 u'Adam Carolla',
 u'Adam Carolla',
 u'Adam Carolla',
 u'Adam Carolla',
 u'Adam Carolla',
 u'Adam Carolla',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Aria Giovanni',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Cutie Mui',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Shin Won-ho',
 u'Chester Morris',
 u'Chester Morris',
 u'Chester Morris',
 u'Chester Morris',
 u'Chester Morris',
 u'Chester Morris',
 u'Chester Morris',
 u'Alice Cooper',
 u'Alice Cooper',
 u'Alice Cooper',

In [80]:
def encoding2name(f_encodings):
        return neigh.predict([f_encodings]) #TODO: largest face encoding

def face2name(img_path):
#TODO: make classes be names instead of numbers
    try:
        img = face_recognition.load_image_file(img_path)
    except:
        return 'failed_to_load'
    bboxes = face_recognition.face_locations(img,model='hog') #TODO: Test if works well else take cnn
    
    if len(bboxes)>1:
        bboxes = [max(bboxes, key=lambda x: ((x[2]-x[0])*(x[1]-x[3])))] #If multiple faces are found, we take the largest face
    elif len(bboxes)==0:
        return 'no_face_found'
    enc = face_recognition.face_encodings(img, bboxes)
#     print(enc[0].shape)
    return encoding2name(enc[0])

In [81]:
for amyGrant in os.listdir('MS_Celeb_uncropped/MS-Celeb-1M/data/croped_face_images/extracted_images/Amy Grant/'):
    print(face2name('MS_Celeb_uncropped/MS-Celeb-1M/data/croped_face_images/extracted_images/Amy Grant/{}'.format(amyGrant)))

[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
no_face_found
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Dannah Phirman']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Nikki Finke']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Gillian Flynn']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy Grant']
[u'Kristiane Backer']
[u'Amy Grant']
[u'Amy Grant']
[u'Amy G